In [ ]:
import include_utils as u
from hdfstorehelper import HDFStoreHelper
import dill
from IPython.parallel import Client
import stopwatch
import rpy2.robjects as robjects
import pandas.rpy.common as com
from pandas import DataFrame, Series
import numpy as np

In [ ]:
hdf = HDFStoreHelper("gypsy_samtools12.hd5")

In [ ]:
pca_data = hdf.get('pca_std_data_snps')

In [ ]:
pca_data[0:5]

In [ ]:
rc = Client(profile="moth")

In [ ]:
dview, lview = u.get_views(rc)
len(dview)

In [ ]:
def do_pca(col):
    try:
        import os, sys, socket, stopwatch
        os.environ['R_HOME'] = '/home/cfriedline/R3/lib64/R'
        os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'])
        import rpy2.robjects as robjects
        import pandas.rpy.common as com
        r = robjects.r
        t = stopwatch.Timer()
        sys.path.append("/home/cfriedline/ipynb/gypsy_moth/")
        print socket.gethostname()
        from hdfstorehelper import HDFStoreHelper
        hdf = HDFStoreHelper("/data7/cfriedline/ipython/notebooks/gypsy_moth/gypsy_samtools12.hd5")
        d = hdf.get('pca_std_data_snps')
        d2 = d.drop(col, axis=1)
        prcomp = r('prcomp')
        res = prcomp(com.convert_to_r_matrix(d2), scale=False, center=False)
        x = com.convert_robj(res.rx2("x"))
        path = "price_in/%s" % col
        t.stop()
        return socket.gethostname(), path, str(t), x.to_dict()
    except Exception as e:
        return "ERROR"

In [ ]:
dview['do_pca'] = do_pca

In [ ]:
jobs = dview.map_async(do_pca, pca_data.columns)